## Assignment 3 - Intrusion Detection System

In [0]:
#importing packages
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.rdd import RDD

In [0]:
#instantiate the spark session
spark = SparkSession.builder.appName("Assignment3").getOrCreate()

## Question 2: Import

In [0]:
#extracting KDD Cup 99 data 
import urllib.request
urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "/tmp/kddcup_data.gz")
dbutils.fs.mv("file:/tmp/kddcup_data.gz", "dbfs:/kdd/kddcup_data.gz")
display(dbutils.fs.ls("dbfs:/kdd"))

path,name,size
dbfs:/kdd/kddcup_data.gz,kddcup_data.gz,2144903


## Question 3: Loading Data & 10 values

In [0]:
#loading data into RDD 
sourceRDD_A3 = spark.sparkContext.textFile("/kdd/kddcup_data.gz", 4)

In [0]:
#Printing 10 values of RDD
sourceRDD_A3.take(10)

Out[119]: ['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,59,59,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,212,1940,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.00,0.00,0.00,0.00,1.00,0.00,1.00,1,69,1.00,0.00,1.00,0.04,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,159,4087,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.00,0.00,0.00,0.00,1.00,0.00,0.00,11,79,1.00,0.00,0.09,0.04,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,210,151,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,8,89,1.00,0.00,0.12,0.04,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,212,786,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,8,99,1.00,0.00,0.12,0.05,0.00,0.00,0.00,0.00,normal.']

In [0]:
#Verifying the type of data structure (RDD)
isinstance(sourceRDD_A3, RDD)

Out[120]: True

## Question 4: Splitting Data

In [0]:
#Splitting the data and printing the results
arrayRDD_A3 = sourceRDD_A3.map(lambda x: x.split(","))
print(arrayRDD_A3.take(1))

[['0', 'tcp', 'http', 'SF', '181', '5450', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '8', '8', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '9', '9', '1.00', '0.00', '0.11', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.']]

In [0]:
#total number of features (columns)
len(arrayRDD_A3.take(1)[0])

Out[122]: 42

## Question 5: Dataframe

In [0]:
#Building a dataframe with corresponding column headers 
df = arrayRDD_A3.toDF(["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate", "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label_s"])

In [0]:
display(df)

duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label_s
0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,59,59,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,212,1940,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.00,0.00,0.00,0.00,1.00,0.00,1.00,1,69,1.00,0.00,1.00,0.04,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,159,4087,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.00,0.00,0.00,0.00,1.00,0.00,0.00,11,79,1.00,0.00,0.09,0.04,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,210,151,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,8,89,1.00,0.00,0.12,0.04,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,212,786,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,8,99,1.00,0.00,0.12,0.05,0.00,0.00,0.00,0.00,normal.


In [0]:
#extracting the columns: duration, protocol_type, service, src_bytes, dst_bytes, flag and label
#displaying 10 values
cols = ['duration','protocol_type','service','src_bytes','dst_bytes','flag','label_s']
new_df = df.select(*cols)
new_df.show(10)

+--------+-------------+-------+---------+---------+----+-------+
duration|protocol_type|service|src_bytes|dst_bytes|flag|label_s|
+--------+-------------+-------+---------+---------+----+-------+
 0| tcp| http| 181| 5450| SF|normal.|
 0| tcp| http| 239| 486| SF|normal.|
 0| tcp| http| 235| 1337| SF|normal.|
 0| tcp| http| 219| 1337| SF|normal.|
 0| tcp| http| 217| 2032| SF|normal.|
 0| tcp| http| 217| 2032| SF|normal.|
 0| tcp| http| 212| 1940| SF|normal.|
 0| tcp| http| 159| 4087| SF|normal.|
 0| tcp| http| 210| 151| SF|normal.|
 0| tcp| http| 212| 786| SF|normal.|
+--------+-------------+-------+---------+---------+----+-------+
only showing top 10 rows

In [0]:
#changing the data type for duration, src_bytes, and dst_bytes to integer 
from pyspark.sql.types import IntegerType
new_df = new_df\
                .withColumn("duration", new_df["duration"].cast(IntegerType()))\
                .withColumn("src_bytes", new_df["src_bytes"].cast(IntegerType()))\
                .withColumn("dst_bytes", new_df["dst_bytes"].cast(IntegerType()))

In [0]:
 #printing Schema 
 new_df.printSchema()

root
-- duration: integer (nullable = true)
-- protocol_type: string (nullable = true)
-- service: string (nullable = true)
-- src_bytes: integer (nullable = true)
-- dst_bytes: integer (nullable = true)
-- flag: string (nullable = true)
-- label_s: string (nullable = true)

In [0]:
#new RDD with the columns: duration, protocol_type, service, src_bytes, dst_bytes, flag and label
#displaying 10 values
RDD_new = new_df.rdd.map(list)
RDD_new.take(10)

Out[128]: [[0, 'tcp', 'http', 181, 5450, 'SF', 'normal.'],
 [0, 'tcp', 'http', 239, 486, 'SF', 'normal.'],
 [0, 'tcp', 'http', 235, 1337, 'SF', 'normal.'],
 [0, 'tcp', 'http', 219, 1337, 'SF', 'normal.'],
 [0, 'tcp', 'http', 217, 2032, 'SF', 'normal.'],
 [0, 'tcp', 'http', 217, 2032, 'SF', 'normal.'],
 [0, 'tcp', 'http', 212, 1940, 'SF', 'normal.'],
 [0, 'tcp', 'http', 159, 4087, 'SF', 'normal.'],
 [0, 'tcp', 'http', 210, 151, 'SF', 'normal.'],
 [0, 'tcp', 'http', 212, 786, 'SF', 'normal.']]

## Question 6: Connections

In [0]:
#Total number of connectiosn based on the protocol_type
protocol_connection = df.groupBy('protocol_type').count().orderBy('count', ascending=False)
display(protocol_connection)

protocol_type,count
icmp,283602
tcp,190065
udp,20354


In [0]:
#Bar graph of protocol_type connection
display(protocol_connection)

protocol_type,count
icmp,283602
tcp,190065
udp,20354


In [0]:
#Total number of connections based on the service
service_connection = df.groupBy('service').count().orderBy('count', ascending=False)
display(service_connection)

service,count
ecr_i,281400
private,110893
http,64293
smtp,9723
other,7237
domain_u,5863
ftp_data,4721
eco_i,1642
ftp,798
finger,670


In [0]:
#bar graph of service connections
display(service_connection)

service,count
ecr_i,281400
private,110893
http,64293
smtp,9723
other,7237
domain_u,5863
ftp_data,4721
eco_i,1642
ftp,798
finger,670


## Question 7: Exploratory Data Analysis

In [0]:
display(new_df.describe())

summary,duration,protocol_type,service,src_bytes,dst_bytes,flag,label_s
count,494021,494021,494021,494021,494021,494021,494021
mean,47.97930249928647,null,null,3025.6102959185946,868.5324247349809,null,null
stddev,707.7464723053699,null,null,988218.1010504081,33040.00125210252,null,null
min,0,icmp,IRC,0,0,OTH,back.
max,58329,udp,whois,693375640,5155468,SH,warezmaster.


#####  Connections

In [0]:
#plotting the distribution of original labels
label_connection = new_df.groupBy('label_s').count().orderBy('count', ascending=False)
display(label_connection)

label_s,count
smurf.,280790
neptune.,107201
normal.,97278
back.,2203
satan.,1589
ipsweep.,1247
portsweep.,1040
warezclient.,1020
teardrop.,979
pod.,264


#####  Replacing everything other than "normal" with "attack"

In [0]:
#replacing all label values that arent "normal" to attack
newDf = new_df.withColumn('label_s', regexp_replace('label_s', 'smurf.|neptune.|back.|satan.|ipsweep.|portsweep.|warezclient.|teardrop.|pod.|nmap.|guess_passwd.|buffer overflow.|buffer_overflow.|land.|warezmaster.|imap.|rootkit.|loadmodule.|ftp_write.|multihop.|phf.|perl.|spy.', 'attack'))

#replacing "normal." with "normal"
newDf = newDf.withColumn('label_s', regexp_replace('label_s', 'normal.', 'normal'))
cols = newDf.columns
display(newDf)

duration,protocol_type,service,src_bytes,dst_bytes,flag,label_s
0,tcp,http,181,5450,SF,normal
0,tcp,http,239,486,SF,normal
0,tcp,http,235,1337,SF,normal
0,tcp,http,219,1337,SF,normal
0,tcp,http,217,2032,SF,normal
0,tcp,http,217,2032,SF,normal
0,tcp,http,212,1940,SF,normal
0,tcp,http,159,4087,SF,normal
0,tcp,http,210,151,SF,normal
0,tcp,http,212,786,SF,normal


#####  Balance of Data

In [0]:
#checking the balance of the data 
labels = newDf.groupBy('label_s').count().orderBy('count', ascending=False)
display(labels)

label_s,count
attack,396743
normal,97278


In [0]:
#bar graph of the counts of attack vs normal
#dataset is unblanaced 
display(labels)

label_s,count
attack,396743
normal,97278


#####  Connections (Attack or Normal) by Service

In [0]:
#Pivot table for service by labels
pivotDF = newDf.groupBy("service").pivot("label_s").count()\
            .na.fill(value=0) \
            .withColumn('total', expr("attack + normal")) \
            .withColumn('attack_perc', expr("(attack/total)*100")) \
            .withColumn('normal_perc', expr("(normal/total)*100")) \
            .withColumn('total_perc', expr("(total/494021)*100")) \
            .sort(col("total").desc())
display(pivotDF)

service,attack,normal,total,attack_perc,normal_perc,total_perc
ecr_i,281055,345,281400,99.8773987206823,0.12260127931769724,56.96114132800023
private,103527,7366,110893,93.35756089203106,6.6424391079689435,22.447021482892428
http,2407,61886,64293,3.743797925124041,96.25620207487596,13.014224091688408
smtp,125,9598,9723,1.2856114367993416,98.71438856320066,1.9681349578256795
other,1605,5632,7237,22.17769794113583,77.82230205886417,1.4649174832648815
domain_u,1,5862,5863,0.017056114617090227,99.98294388538291,1.186791654605776
ftp_data,923,3798,4721,19.550942596907433,80.44905740309257,0.9556273923578147
eco_i,1253,389,1642,76.30937880633374,23.69062119366626,0.3323745346857725
ftp,425,373,798,53.258145363408524,46.741854636591476,0.16153159481074691
finger,202,468,670,30.149253731343283,69.85074626865672,0.1356217650666672


In [0]:
#pie charts for % of service for all connections, normal, and attack. 
display(pivotDF.take(5))

service,attack,normal,total,attack_perc,normal_perc,total_perc
ecr_i,281055,345,281400,99.8773987206823,0.12260127931769724,56.96114132800023
private,103527,7366,110893,93.35756089203106,6.6424391079689435,22.447021482892428
http,2407,61886,64293,3.743797925124041,96.25620207487596,13.014224091688408
smtp,125,9598,9723,1.2856114367993416,98.71438856320066,1.9681349578256795
other,1605,5632,7237,22.17769794113583,77.82230205886417,1.4649174832648815


In [0]:
#Bar graph showing the top 5 service by connection (attack/normal)
display(pivotDF.take(5))

service,attack,normal,total,attack_perc,normal_perc,total_perc
ecr_i,281055,345,281400,99.8773987206823,0.12260127931769724,56.96114132800023
private,103527,7366,110893,93.35756089203106,6.6424391079689435,22.447021482892428
http,2407,61886,64293,3.743797925124041,96.25620207487596,13.014224091688408
smtp,125,9598,9723,1.2856114367993416,98.71438856320066,1.9681349578256795
other,1605,5632,7237,22.17769794113583,77.82230205886417,1.4649174832648815


#####  Connections (Attack or Normal) by Flag

In [0]:
#Flag - normal or error status of the connection 
#Pivot table for flag by labels
pivotDF_2 = newDf.groupBy("flag").pivot("label_s").count()\
            .na.fill(value=0) \
            .withColumn('total', expr("attack + normal")) \
            .withColumn('total_perc', expr("(total/494021)*100")) \
            .sort(col("total").desc())
display(pivotDF_2)

flag,attack,normal,total,total_perc
SF,286731,91709,378440,76.60403100273065
S0,86956,51,87007,17.612004347993306
REJ,21534,5341,26875,5.44005214353236
RSTR,872,31,903,0.1827857520226873
RSTO,512,67,579,0.11720149548298554
SH,107,0,107,0.02165899830169163
S1,3,54,57,0.011537971057910493
S2,7,17,24,0.004858093077014945
RSTOS0,11,0,11,0.0022266259936318494
S3,3,7,10,0.002024205448756227


In [0]:
#pie chart showing the % of top 3 flag for all connections
display(pivotDF_2.take(3))

flag,attack,normal,total,total_perc
SF,286731,91709,378440,76.60403100273065
S0,86956,51,87007,17.612004347993306
REJ,21534,5341,26875,5.44005214353236


In [0]:
#Bar graph showing the top 3 flag by connection (attack/normal)
display(pivotDF_2.take(3))

flag,attack,normal,total,total_perc
SF,286731,91709,378440,76.60403100273065
S0,86956,51,87007,17.612004347993306
REJ,21534,5341,26875,5.44005214353236


#####  Connections (Attack or Normal) by Protocol Type

In [0]:
#Pivot table for protocol type by labels
pivotDF_3 = newDf.groupBy("protocol_type").pivot("label_s").count()\
            .na.fill(value=0) \
            .withColumn('total', expr("attack + normal")) \
            .withColumn('total_perc', expr("(total/494021)*100")) \
            .sort(col("total").desc())
display(pivotDF_3)

protocol_type,attack,normal,total,total_perc
icmp,282314,1288,283602,57.40687136781635
tcp,113252,76813,190065,38.47306086178523
udp,1177,19177,20354,4.120067770398425


In [0]:
#pie chart showing the % of protocol type for all connections
display(pivotDF_3)

protocol_type,attack,normal,total,total_perc
icmp,282314,1288,283602,57.40687136781635
tcp,113252,76813,190065,38.47306086178523
udp,1177,19177,20354,4.120067770398425


## Question 8: Model

#######  Replacing everything other than "normal" with "attack" - Answered in Question 7

##### Data Preprocessing

In [0]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.stat import Correlation

In [0]:
#preparing data 
#Reference: https://docs.databricks.com/applications/machine-learning/train-model/mllib/index.html#binary-classification-example
categoricalColumns = ['protocol_type', 'service', 'flag']
stages = []

#OneHotEncoding all categorial columns 
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

#Encoding label column
label_stringIdx = StringIndexer(inputCol = 'label_s', outputCol = 'label')
stages += [label_stringIdx]

numericCols = ['duration', 'src_bytes', 'dst_bytes']

#Vectorizing
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

stages += [assembler]

In [0]:
#creating the pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(newDf)
newDf_1 = pipelineModel.transform(newDf)

newDf_1.printSchema()

root
-- duration: integer (nullable = true)
-- protocol_type: string (nullable = true)
-- service: string (nullable = true)
-- src_bytes: integer (nullable = true)
-- dst_bytes: integer (nullable = true)
-- flag: string (nullable = true)
-- label_s: string (nullable = true)
-- protocol_typeIndex: double (nullable = false)
-- protocol_typeclassVec: vector (nullable = true)
-- serviceIndex: double (nullable = false)
-- serviceclassVec: vector (nullable = true)
-- flagIndex: double (nullable = false)
-- flagclassVec: vector (nullable = true)
-- label: double (nullable = false)
-- features: vector (nullable = true)

In [0]:
display(newDf_1)

duration,protocol_type,service,src_bytes,dst_bytes,flag,label_s,protocol_typeIndex,protocol_typeclassVec,serviceIndex,serviceclassVec,flagIndex,flagclassVec,label,features
0,tcp,http,181,5450,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 181.0, 5450.0))"
0,tcp,http,239,486,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 239.0, 486.0))"
0,tcp,http,235,1337,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 235.0, 1337.0))"
0,tcp,http,219,1337,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 219.0, 1337.0))"
0,tcp,http,217,2032,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 217.0, 2032.0))"
0,tcp,http,217,2032,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 217.0, 2032.0))"
0,tcp,http,212,1940,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 212.0, 1940.0))"
0,tcp,http,159,4087,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 159.0, 4087.0))"
0,tcp,http,210,151,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 210.0, 151.0))"
0,tcp,http,212,786,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 212.0, 786.0))"


In [0]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(newDf_1)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(newDf_1)
display(scaledData)

duration,protocol_type,service,src_bytes,dst_bytes,flag,label_s,protocol_typeIndex,protocol_typeclassVec,serviceIndex,serviceclassVec,flagIndex,flagclassVec,label,features,scaledFeatures
0,tcp,http,181,5450,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 181.0, 5450.0))","List(0, 80, List(1, 4, 67, 78, 79), List(2.055363006399789, 2.9721197203276812, 2.362126925892916, 1.83157948440338E-4, 0.16495156759878016))"
0,tcp,http,239,486,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 239.0, 486.0))","List(0, 80, List(1, 4, 67, 78, 79), List(2.055363006399789, 2.9721197203276812, 2.362126925892916, 2.4184944573061204E-4, 0.014709442541836176))"
0,tcp,http,235,1337,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 235.0, 1337.0))","List(0, 80, List(1, 4, 67, 78, 79), List(2.055363006399789, 2.9721197203276812, 2.362126925892916, 2.3780175626231728E-4, 0.040466100161388824))"
0,tcp,http,219,1337,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 219.0, 1337.0))","List(0, 80, List(1, 4, 67, 78, 79), List(2.055363006399789, 2.9721197203276812, 2.362126925892916, 2.2161099838913822E-4, 0.040466100161388824))"
0,tcp,http,217,2032,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 217.0, 2032.0))","List(0, 80, List(1, 4, 67, 78, 79), List(2.055363006399789, 2.9721197203276812, 2.362126925892916, 2.1958715365499084E-4, 0.06150120832306813))"
0,tcp,http,217,2032,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 217.0, 2032.0))","List(0, 80, List(1, 4, 67, 78, 79), List(2.055363006399789, 2.9721197203276812, 2.362126925892916, 2.1958715365499084E-4, 0.06150120832306813))"
0,tcp,http,212,1940,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 212.0, 1940.0))","List(0, 80, List(1, 4, 67, 78, 79), List(2.055363006399789, 2.9721197203276812, 2.362126925892916, 2.145275418196224E-4, 0.05871670479663))"
0,tcp,http,159,4087,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 159.0, 4087.0))","List(0, 80, List(1, 4, 67, 78, 79), List(2.055363006399789, 2.9721197203276812, 2.362126925892916, 1.608956563647168E-4, 0.12369854252774579))"
0,tcp,http,210,151,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 210.0, 151.0))","List(0, 80, List(1, 4, 67, 78, 79), List(2.055363006399789, 2.9721197203276812, 2.362126925892916, 2.1250369708547502E-4, 0.004570217744479964))"
0,tcp,http,212,786,SF,normal,1.0,"List(0, 2, List(1), List(1.0))",2.0,"List(0, 65, List(2), List(1.0))",0.0,"List(0, 10, List(0), List(1.0))",1.0,"List(0, 80, List(1, 4, 67, 78, 79), List(1.0, 1.0, 1.0, 212.0, 786.0))","List(0, 80, List(1, 4, 67, 78, 79), List(2.055363006399789, 2.9721197203276812, 2.362126925892916, 2.145275418196224E-4, 0.023789345345438754))"


##### Split Data

In [0]:
#splitting the data into training and testing data splits, 70% training, 30% testing
train, test = scaledData.randomSplit([0.7, 0.3])
print("Training Dataset Shape: (" + str(train.count())+ "," + str(len(train.columns)) + ")")
print("Testing Dataset Shape: (" + str(test.count())+ "," + str(len(test.columns)) + ")")

Training Dataset Shape: (345970,16)
Testing Dataset Shape: (148051,16)

##### Machine Learning Model: Intrusion Detection

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.evaluation import BinaryClassificationEvaluator

#Evaluation with Area Under Curve
evaluator = BinaryClassificationEvaluator()

In [0]:
#Creating LogisticRegression model
lr = LogisticRegression(featuresCol = 'scaledFeatures', labelCol = 'label', maxIter=10)

#Train model with Training Data 
lrModel = lr.fit(train)
#Model predictions on Testing Data
predictions = lrModel.transform(test)

#print('Test Area Under ROC', evaluator.evaluate(predictions))
print("AUROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))
print("AUPR: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})))

AUROC: 0.9960835481784368
AUPR: 0.9897716833128777

In [0]:
#Creating Random Forest Classifier 
rf = RandomForestClassifier(featuresCol = 'scaledFeatures', labelCol = 'label')

#Train Model with Training Data
rfModel = rf.fit(train)
#Model predicitons on Testing Data
predictions = rfModel.transform(test)

print("AUROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))
print("AUPR: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})))

AUROC: 0.9992464841620312
AUPR: 0.9958464144664521